In [3]:
%load_ext autoreload
%autoreload 2
import logging
logging.basicConfig(level=logging.INFO, format='%(levelname)s: %(message)s')

logger = logging.getLogger()

In [5]:
from LCNE_patchseq_analysis.data_util.metadata import read_brian_spreadsheet, cross_check_metadata

##  Load patchseq metadata and perform cross check

In [6]:
# If run in CO, we cannot access LIMS
dfs = read_brian_spreadsheet(file_path="/root/capsule/data/IVSCC_LC_summary.xlsx", add_lims=False)
for source in ["tab_xyz", "tab_ephys_fx"]:
    df_inconsistencies = cross_check_metadata(dfs["df_merged"], source)

INFO: Reading metadata from /root/capsule/data/IVSCC_LC_summary.xlsx...


INFO: 
INFO: --------------------------------------------------
INFO: Cross-checking metadata between tab_xyz and master tables...
INFO: Source columns: ['x_tab_xyz', 'y_tab_xyz', 'z_tab_xyz', 'Annotated structure_tab_xyz', 'notes_tab_xyz']
INFO: Master columns: ['x_tab_master', 'y_tab_master', 'z_tab_master', 'Annotated structure_tab_master', 'notes_tab_master']
2023-09-01 Dbh-Cre_KH212;RCL-H2B-GFP-692026.10.10.02  10534.982420 10151.01953
2023-08-20 Dbh-Cre_KH212;RCL-H2B-GFP-692023.08.06.01  10541.875980 10702.28320
2023-08-20 Dbh-Cre_KH212;RCL-H2B-GFP-692023.08.06.02  10702.283200 10761.00195
2023-06-02 Dbh-Cre_KH212;RCL-H2B-GFP-676766.10.06.03  10521.757810 10541.87598
2023-03-15                   C57BL6J-665266.11.06.03  10451.809570 10534.98242
2023-01-20     Ndnf-IRES2-dgCre;Ai14-659663.11.06.03  10391.497070 10521.75781
2023-01-20     Ndnf-IRES2-dgCre;Ai14-659663.11.06.04   9531.198242 10451.80957
2023-09-01 Dbh-Cre_KH212;RCL-H2B-GFP-692026.10.10.02   4183.531250 3701.974609
20

### ❌ Oh no! These inconsistencies must be caused by manually copying and pasting across the tabs!!!

## Quick overview using pygwalker

In [14]:
!pip install pygwalker --quiet

In [15]:
import pygwalker as pyg
walker = pyg.walk(df)
walker

NameError: name 'df' is not defined

## Extract AP

In [9]:
df_merged, df_tab_ephys_fx = dfs["df_merged"], dfs["df_tab_ephys_fx"]
df_merged.columns

Index(['Date', 'donor', 'jem-id_cell_specimen', 'cell_specimen_id',
       'type of experiment', 'injection region', 'virus',
       'jem-status_reporter', 'jem-id_patched_cell_container',
       'jem-id_patched_cell_container.1',
       ...
       'peak_deltav_hero', 'first_isi_inv_rheo', 'first_isi_inv_hero',
       'upstroke_downstroke_ratio_short_square', 'threshold_v_short_square',
       'width_short_square', 'upstroke_short_square',
       'downstroke_short_square', 'width_suprathresh_short_square',
       'peak_deltav_short_square'],
      dtype='object', length=131)

In [10]:
[col for col in df_merged.columns if "roi" in col]

['jem-roi_minor', 'ephys_roi_id']

In [11]:
[col for col in df_tab_ephys_fx.columns if "rheo" in col]

['rheobase_i',
 'latency_rheo',
 'avg_rate_rheo',
 'mean_isi_rheo',
 'upstroke_downstroke_ratio_rheo',
 'threshold_v_rheo',
 'peak_v_rheo',
 'width_rheo',
 'upstroke_rheo',
 'downstroke_rheo',
 'width_suprathresh_rheo',
 'trough_v_rheo',
 'fast_trough_v_rheo',
 'postap_slope_rheo',
 'fast_trough_v_last_rheo',
 'adp_v_last_rheo',
 'trough_slowdeltav_rheo',
 'fast_trough_deltav_rheo',
 'peak_deltav_rheo',
 'first_isi_inv_rheo']

In [12]:
[col for col in df_tab_ephys_fx.columns if "hero" in col]

['adapt_hero',
 'avg_rate_hero',
 'latency_hero',
 'mean_isi_hero',
 'median_isi_hero',
 'isi_cv_hero',
 'ir_late_hero',
 'upstroke_downstroke_ratio_hero',
 'threshold_v_hero',
 'peak_v_hero',
 'width_hero',
 'upstroke_hero',
 'downstroke_hero',
 'width_suprathresh_hero',
 'trough_v_hero',
 'fast_trough_v_hero',
 'postap_slope_hero',
 'ahp_delay_ratio_hero',
 'trough_slowdeltav_hero',
 'fast_trough_deltav_hero',
 'peak_deltav_hero',
 'first_isi_inv_hero']

## Load one nwb

In [13]:
RAW_DIRECTORY = "/root/capsule/data/LCNE-patchseq-ephys-raw"
ephys_roi_id = "1410790193"
raw_path_this = f"{RAW_DIRECTORY}/Ephys_Roi_Result_{ephys_roi_id}"

In [14]:
import glob
nwbs = glob.glob(f"{raw_path_this}/*.nwb")
jsons = glob.glob(f"{raw_path_this}/*.json")
nwbs

['/root/capsule/data/LCNE-patchseq-ephys-raw/Ephys_Roi_Result_1410790193/1410790193_1410793600_spikes.nwb',
 '/root/capsule/data/LCNE-patchseq-ephys-raw/Ephys_Roi_Result_1410790193/Dbh-Cre-KI;Ai65-773002.06.01.01.nwb']

In [15]:
[j for j in jsons if "input" in j]

['/root/capsule/data/LCNE-patchseq-ephys-raw/Ephys_Roi_Result_1410790193/EPHYS_QC_V3_QUEUE_1410790193_input.json',
 '/root/capsule/data/LCNE-patchseq-ephys-raw/Ephys_Roi_Result_1410790193/EPHYS_FEATURE_EXTRACTION_V3_QUEUE_1410790193_input.json',
 '/root/capsule/data/LCNE-patchseq-ephys-raw/Ephys_Roi_Result_1410790193/EPHYS_NWB_STIMULUS_SUMMARY_V3_QUEUE_1410790193_input.json']

In [16]:
[j for j in jsons if "output" in j]

['/root/capsule/data/LCNE-patchseq-ephys-raw/Ephys_Roi_Result_1410790193/202412181106_EPHYS_NWB_STIMULUS_SUMMARY_V3_QUEUE_1410790193_output.json',
 '/root/capsule/data/LCNE-patchseq-ephys-raw/Ephys_Roi_Result_1410790193/202412181110_EPHYS_QC_V3_QUEUE_1410790193_output.json',
 '/root/capsule/data/LCNE-patchseq-ephys-raw/Ephys_Roi_Result_1410790193/202412181120_EPHYS_FEATURE_EXTRACTION_V3_QUEUE_1410790193_output.json']

In [17]:
from pynwb import NWBHDF5IO

nwb = NWBHDF5IO(nwbs[0], mode='r').read()

/opt/conda/lib/python3.9/site-packages/hdmf/spec/namespace.py:535: UserWarning: Ignoring cached namespace 'hdmf-common' version 1.1.3 because version 1.8.0 is already loaded.
  warn("Ignoring cached namespace '%s' version %s because version %s is already loaded."
/opt/conda/lib/python3.9/site-packages/hdmf/spec/namespace.py:535: UserWarning: Ignoring cached namespace 'core' version 2.2.4 because version 2.8.0 is already loaded.
  warn("Ignoring cached namespace '%s' version %s because version %s is already loaded."
/opt/conda/lib/python3.9/site-packages/pynwb/core.py:56: UserWarning: Use of icephys_filtering is deprecated and will be removed in PyNWB 4.0. Use the IntracellularElectrode.filtering field instead
  warn(error_msg)


In [57]:
import h5py

hdf = h5py.File(nwbs[0], 'r')
acquisition = hdf.attrs['acquisition']

KeyError: "Unable to synchronously open attribute (can't locate attribute: 'acquisition')"

In [53]:
list(nwb.fields.keys())

['session_description',
 'identifier',
 'session_start_time',
 'timestamps_reference_time',
 'file_create_date',
 'icephys_electrodes',
 'acquisition',
 'stimulus',
 'devices',
 'intervals',
 'subject',
 'sweep_table',
 'epochs',
 'data_collection',
 'source_script',
 'source_script_file_name',
 'icephys_filtering']

In [51]:
nwb.acquisition["data_00074_AD0"].sweep_number

np.uint32(74)

In [60]:
list(nwb.acquisition["data_00000_AD0"].fields.keys())

['starting_time',
 'rate',
 'resolution',
 'comments',
 'description',
 'conversion',
 'offset',
 'unit',
 'data',
 'starting_time_unit',
 'electrode',
 'gain',
 'stimulus_description',
 'sweep_number',
 'capacitance_fast',
 'capacitance_slow']

In [ ]:
import panel as pn
import matplotlib.pyplot as plt
import numpy as np

# Initialize Panel extension
pn.extension()

# Create slider widget
n_slider = pn.widgets.IntSlider(name='Sweep number', start=0, end=74, value=25)

# Define plotting function
def plot_raw(n):
    fig, ax = plt.subplots(figsize=(6, 4))
    ax.plot(np.array(nwb.acquisition[f"data_{n:05}_AD0"].data))
    ax.set_title(f'Sweep number {n}')
    plt.close(fig)  # Prevents duplicate display
    return fig

# Use the @pn.depends decorator to update the plot whenever the slider value changes
@pn.depends(n_slider)
def update_plot(n):
    return plot_raw(n)

# Arrange the slider and plot in a vertical layout
app = pn.Column(n_slider, update_plot)
app

<IPython.core.display.Javascript object>

BokehModel(combine_events=True, render_bundle={'docs_json': {'75951405-1800-47ed-a22e-37b94eb8ed67': {'version…

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>